In [1]:
# Import modules.
import torch
import random

import numpy as np

from game import Game
from network import H_GO
from torch import nn, optim
from torchsummary import summary
from dataReader import DataReader
from trainer import SL_Trainer, RL_Trainer

In [2]:
# Set training device.
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available.")
else:
    device = torch.device("cpu")
    print("CUDA is not available, using CPU instead.")

CUDA is available.


In [3]:
# Set random seed.
random_seed = 0

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [4]:
# Set hyperparameters.
epochs = 5

batch_size = 256
data_num = 500
data_augment = False

In [5]:
# Load data.
data_reader = DataReader(dir_path="../dataset/pro", load_num=data_num, train_ratio=0.8, augment=data_augment)

Read raw data completed. (0.39 s)
Clear data completed. (0.01 s)
Convert data completed. (0.03 s)
Make train data completed. (0.91 s)
Split train and test data completed. (68359 train data, 17090 test data).


In [6]:
# set model.
model = H_GO(input_size=8, output_size=361, hidden_dim=64)
model.to(device=device)

H_GO(
  (conv1): Conv2d(8, 64, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (tanh): Tanh()
  (layers): Sequential(
    (0): Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): Block(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    

In [7]:
# Print model summary.
summary(model, input_size=(8, 19, 19))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 17, 17]           4,672
       BatchNorm2d-2           [-1, 64, 17, 17]             128
              Tanh-3           [-1, 64, 17, 17]               0
            Conv2d-4           [-1, 64, 17, 17]          36,928
       BatchNorm2d-5           [-1, 64, 17, 17]             128
              ReLU-6           [-1, 64, 17, 17]               0
            Conv2d-7           [-1, 64, 17, 17]          36,928
       BatchNorm2d-8           [-1, 64, 17, 17]             128
              ReLU-9           [-1, 64, 17, 17]               0
            Block-10           [-1, 64, 17, 17]               0
           Conv2d-11           [-1, 64, 17, 17]          36,928
      BatchNorm2d-12           [-1, 64, 17, 17]             128
             ReLU-13           [-1, 64, 17, 17]               0
           Conv2d-14           [-1, 64,

In [8]:
# Set optimizer and loss function.
optimizer = optim.RAdam(params=model.parameters())
loss_fu_policy = nn.CrossEntropyLoss()
loss_fu_value = nn.BCELoss()

In [9]:
sl_trainer = SL_Trainer(epochs=epochs, model=model, device=device, optimizer=optimizer, loss_fu_policy=loss_fu_policy, loss_fu_value=loss_fu_value)
sl_trainer.train(data_reader=data_reader, batch_size=batch_size, test_every_epoch=5)

Epoch: 000 | Loss of model: 1608.197 | Accuracy of policy: 1.024% | Accuracy of value: 83.745%
Accuracy of policy: 0.960% | Accuracy of value: 55.600%
Epoch: 001 | Loss of model: 1448.182 | Accuracy of policy: 2.427% | Accuracy of value: 95.679%
Epoch: 002 | Loss of model: 1275.910 | Accuracy of policy: 5.040% | Accuracy of value: 96.115%
Epoch: 003 | Loss of model: 1189.501 | Accuracy of policy: 7.446% | Accuracy of value: 96.511%
Epoch: 004 | Loss of model: 1135.334 | Accuracy of policy: 9.351% | Accuracy of value: 96.804%


In [10]:
sl_trainer.test(data_reader=data_reader, batch_size=batch_size)

Accuracy of policy: 8.344% | Accuracy of value: 53.938%


In [11]:
game = Game(size=19, model=model, device=device)
game.play()

e:\Project\H-Go\src\hts.py:76: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:233.)
  input = torch.tensor([input], dtype=torch.float).to(self.device)


AI's move: (11, 2)
  A B C D E F G H I J K L M N O P Q R S
A · · · · · · · · · · · · · · · · · · · 
B · · · · · · · · · · · · · · · · · · · 
C · · · · · · · · · · · · · · · · · · · 
D · · · · · · · · · · · · · · · · · · · 
E · · · · · · · · · · · · · · · · · · · 
F · · · · · · · · · · · · · · · · · · · 
G · · · · · · · · · · · · · · · · · · · 
H · · · · · · · · · · · · · · · · · · · 
I · · · · · · · · · · · · · · · · · · · 
J · · · · · · · · · · · · · · · · · · · 
K · · · · · · · · · · · · · · · · · · · 
L · ·(X)· · · · · · · · · · · · · · · · 
M · · · · · · · · · · · · · · · · · · · 
N · · · · · · · · · · · · · · · · · · · 
O · · · · · · · · · · · · · · · · · · · 
P · · · · · · · · · · · · · · · · · · · 
Q · · · · · · · · · · · · · · · · · · · 
R · · · · · · · · · · · · · · · · · · · 
S · · · · · · · · · · · · · · · · · · · 

Player's move: (11, 3)
  A B C D E F G H I J K L M N O P Q R S
A · · · · · · · · · · · · · · · · · · · 
B · · · · · · · · · · · · · · · · · · · 
C · · · · · · · 